In [0]:
!pip install -q http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl torchvision

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

import matplotlib.pyplot as plt
from IPython import display
from typing import List, Tuple, Dict

# Машинный перевод

В предыдущих сериях мы смотрели на примеры использования рекуррентных нейронных сетей для классификации последовательностей (предсказание языка по фамилии), для разметки последовательностей (POS-Tagging) и генерации последовательностей.

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg =x250)

Последний популярный вариант применения - seq2seq: по сути, это объединение архитектур для классификации и для генерации последовательностей.

Самое известное применение seq2seq - для машинного перевода:   
![MT](https://www.tensorflow.org/images/seq2seq/seq2seq.jpg =x500)
From [Neural Machine Translation (seq2seq) Tutorial](https://www.tensorflow.org/tutorials/seq2seq).

При обработке слов на исходном языке рекуррентная сеть кодирует его содержание в скрытом векторе. 

Дальше этот вектор передается в генератор текста на нужном нам языке в качестве начального скрытого состояния.

Сам генератор устроен так же, как и тот генератор фамилий, который мы уже писали. Но переданное скрытое состояние с закодированным в нем смыслом исходного предложения позволяет ему генерировать текст, имеющий смысл, близкий к нужному.

## Данные

"Взрослые" датасеты можно взять здесь: http://www.statmt.org/wmt16/translation-task.html#download

Мы будем использовать "игрушечный" датасет для en-ru: http://www.manythings.org/anki/rus-eng.zip

In [0]:
!wget http://www.manythings.org/anki/rus-eng.zip 
!unzip rus-eng.zip 

In [0]:
!head rus.txt
!wc -l rus.txt

In [0]:
!shuf rus.txt > rus.shuffled.txt
!head rus.shuffled.txt -n 30000 > rus.val.txt
!tail rus.shuffled.txt -n+30000 > rus.train.txt
!cut -f 1 rus.val.txt > en.val.txt
!cut -f 2 rus.val.txt > ru.val.txt
!cut -f 1 rus.train.txt > en.train.txt
!cut -f 2 rus.train.txt > ru.train.txt

!head ru.train.txt
!head en.train.txt
!wc -l en.train.txt

## Препроцессинг

Начнем с токенизации.

In [0]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

def preprocess_file(input_file_name, output_file_name):
    with open(input_file_name, "r", encoding='utf-8') as i, open(output_file_name, "w", encoding='utf-8') as o:
        for line in i:
            tokens = [token.lower() for token in word_tokenize(line.strip())]
            o.write(" ".join(tokens) + "\n")

preprocess_file("en.val.txt", "en.val.clean.txt")
preprocess_file("ru.val.txt", "ru.val.clean.txt")
preprocess_file("en.train.txt", "en.train.clean.txt")
preprocess_file("ru.train.txt", "ru.train.clean.txt")
!head "en.val.clean.txt"
!head "ru.val.clean.txt"

Составим пару словарей - для исходного и результирующего языка.

In [0]:
from collections import Counter
import os
import pickle


class Vocabulary:
    def __init__(self, language):
        self.language = language
        self.word2index = {}
        self.word2count = Counter()
        self.special_tokens = ("<pad>", "</b>", "</s>", "<unk>")
        self.index2word = list(self.special_tokens)

    def get_pad(self):
        return self.index2word.index("<pad>")

    def get_sos(self):
        return self.index2word.index("</b>")

    def get_eos(self):
        return self.index2word.index("</s>")

    def get_unk(self):
        return self.index2word.index("<unk>")

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            if word == '':
                continue
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = len(self.index2word)
            self.word2count[word] += 1
            self.index2word.append(word)
        else:
            self.word2count[word] += 1

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        else:
            return self.get_unk()
          
    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        return self.size() <= len(self.special_tokens)

    def shrink(self, n):
        best_words = self.word2count.most_common(n)
        self.index2word = list(self.special_tokens)
        self.word2index = {}
        self.word2count = Counter()
        for word, count in best_words:
            self.add_word(word)
            self.word2count[word] = count

    def reset(self):
        self.word2index = {}
        self.word2count = Counter()
        self.index2word = list(self.special_tokens)

    def save(self, path) -> None:
        with open(path, "wb") as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)

    def load(self, path):
        with open(path, "rb") as f:
            vocab = pickle.load(f)
            self.__dict__.update(vocab.__dict__)
            
    def get_indices(self, sentence: str) -> List[int]:
        return [self.get_index(word) for word in sentence.strip().split()] + [self.get_eos()]
      
    def pad_indices(self, indices: List[int], max_length: int):
        return indices + [self.get_pad() for _ in range(max_length - len(indices))]
    
    def add_file(self, filename: str):
        with open(filename, "r", encoding="utf-8") as r:
            for line in r:
                for word in line.strip().split():
                    self.add_word(word)

In [0]:
def collect_vocabularies(src_filename, src_lang, tgt_filename, tgt_lang):
    src_vocabulary = Vocabulary(src_lang)
    tgt_vocabulary = Vocabulary(tgt_lang)
    src_vocabulary.add_file(src_filename)
    src_vocabulary.save("src_vocabulary.pickle")
    tgt_vocabulary.add_file(tgt_filename)
    tgt_vocabulary.save("tgt_vocabulary.pickle")
    return src_vocabulary, tgt_vocabulary
  
SRC_FILENAME = "ru.train.clean.txt"
TGT_FILENAME = "en.train.clean.txt"
SRC_VAL_FILENAME = "ru.val.clean.txt"
TGT_VAL_FILENAME = "en.val.clean.txt"
src_vocabulary, tgt_vocabulary = collect_vocabularies(SRC_FILENAME, "en", TGT_FILENAME, "fr")
print(src_vocabulary.size(), tgt_vocabulary.size())

src_vocabulary.shrink(20000)

Напишем генератор батчей.

In [0]:
class BatchGenerator:
    def __init__(self, 
                 pair_file_names: List[Tuple[str, str]], 
                 max_len: int, 
                 batch_size: int,
                 src_vocabulary: Vocabulary, 
                 tgt_vocabulary: Vocabulary):
        self.pair_file_names = pair_file_names  # type: List[Tuple[str, str]]
        self.max_len = max_len  # type: int
        self.src_vocabulary = src_vocabulary  # type: Vocabulary
        self.tgt_vocabulary = tgt_vocabulary  # type: Vocabulary
        self.batch_size = batch_size
        
    def __iter__(self):
        batch_count = 0
        pairs = list()
        for src_file_name, tgt_file_name in self.pair_file_names:
            with open(src_file_name, "r", encoding='utf-8') as src, open(tgt_file_name, "r", encoding='utf-8') as tgt:
                for src_sentence, tgt_sentence in zip(src, tgt):
                    src_indices = self.src_vocabulary.get_indices(src_sentence)
                    tgt_indices = self.tgt_vocabulary.get_indices(tgt_sentence)
                    if len(src_indices) > self.max_len or len(tgt_indices) > self.max_len:
                        continue
                    pairs.append((src_indices, tgt_indices))
                    if len(pairs) == self.batch_size:
                        src_seqs, tgt_seqs = zip(*pairs)
                        src_batch, tgt_batch = self.process(src_seqs, self.src_vocabulary), \
                                               self.process(tgt_seqs, self.tgt_vocabulary)
                        yield src_batch, tgt_batch
                        pairs = list()
            if len(pairs) == 0:
                continue;
            src_seqs, tgt_seqs = zip(*pairs)
            src_batch, tgt_batch = self.process(src_seqs, self.src_vocabulary), \
                                   self.process(tgt_seqs, self.tgt_vocabulary)
            yield src_batch, tgt_batch

    def process(self, sequences: List[List[int]], vocabulary: Vocabulary):
        lengths = BatchGenerator.get_lengths(sequences)
        sequences = self.pad(sequences, lengths, vocabulary)
        variable = BatchGenerator.get_variable(sequences)
        return variable
    
    def pad(self, sequences: List[List[int]], lengths: List[int], vocabulary: Vocabulary):
        return [vocabulary.pad_indices(indices, max(lengths)) for indices in sequences]
      
    @staticmethod
    def get_lengths(sequences: List[List[int]]):
        return [len(indices) for indices in sequences]

    @staticmethod
    def get_variable(sequences: List[List[int]]):
        return autograd.Variable(torch.LongTensor(sequences), requires_grad=False).transpose(0, 1)

In [0]:
batch_generator = BatchGenerator([(SRC_FILENAME, TGT_FILENAME)], 10, 512, src_vocabulary, tgt_vocabulary)
next(iter(batch_generator))

In [0]:
val_batch_generator = BatchGenerator([(SRC_VAL_FILENAME, TGT_VAL_FILENAME)], 10, 512, src_vocabulary, tgt_vocabulary)

## Seq2Seq

Сама модель повторяет уже описанную выше.

Составим её из нескольких кусков. Во-первых, `EncoderRNN` будет читать предложение на исходном языке и строить его вектор смысла.

**Задание** Реализуйте метод `forward`. Обратите внимание, самое главное, что он возвращает - это `hidden`, а не `outputs`, как обычно.

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout)

    def forward(self, input_seqs, hidden=None):
        <your code here>
        return outputs, hidden

Декодер будет устроен сложнее.

На вход рекуррентной сети внутри него на каждом шаге нужно отдавать предыдущее слово.

Различают два способа, откуда это предыдущее слово может взяться: во-первых, *teacher forcing* - это когда у нас есть предложение, которое мы должны сгенерировать и для генерации следующего слова используется предыдущее слово из этого правильного варианта. В итоге, сеть могла сгенерировать неправильное слово на прошлом шаге - но мы, как строгий учитель, исправляем её вариант на правильный.

Альтернативный вариант - это использовать то слово, которое сгенерировала сеть. Если использовать teacher forcing сеть становится более зависимой от правильности предыдущего слова - она просто привыкает, что предыдущее слово всегда правильное. Поэтому тренировать сеть в условиях, близких к боевым, может быть очень полезным.

Pytorch предоставляет очень удобную возможность сочетать оба подхода: почему бы не тренировать сеть иногда с teacher forcing, иногда - без. Можно случайно выбирать метод для каждого мини-батча - скажем, с 50%-ой вероятностью каждый из вариантов.

Чтобы это работало, добавим в `forward` декодера параметр `gtruth`, который как раз и будет содержать (или не содержать) правильное предложение.

**Задание** Реализуйте класс `Generator`, который будет просто проецировать скрытое состояние декодера на словарь и строить распределение вероятностей предсказаний слов.

Реализуйте метод `step` в декодере, который будет делать один шаг рекуррентной сети (с новым только сгенерированным словом).

Реализуйте сэмплирование из предсказания генератора для случая обучения без teacher forcing'а.

In [0]:
class Generator(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size

        self.out = nn.Linear(hidden_size, output_size)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        assert inputs.size(1) == self.hidden_size
        <your code here>
        return output


class DecoderRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_size, output_size, max_length, n_layers=3, 
                 dropout=0.3, use_cuda=False):
        super(DecoderRNN, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.use_cuda = use_cuda
        self.max_length = max_length

        self.embedding = nn.Embedding(output_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout)
        self.generator = Generator(hidden_size, output_size)
        
    def step(self, batch_input, hidden):       
        <your code here>
        return output, hidden

    def init_state(self, batch_size, sos_index):
        initial_input = autograd.Variable(torch.zeros((batch_size, )).type(torch.LongTensor))
        initial_input = torch.add(initial_input, sos_index)
        initial_input = initial_input.cuda() if self.use_cuda else initial_input
        return initial_input

    def forward(self, current_input, hidden, length, gtruth=None):
        batch_size = current_input.size(0)
        outputs = autograd.Variable(torch.zeros(length, batch_size, self.output_size))
        outputs = outputs.cuda() if self.use_cuda else outputs

        for t in range(length):
            output, hidden = self.step(current_input, hidden)
            scores = self.generator.forward(output.squeeze(0))
            outputs[t] = scores
            if gtruth is None:
                <generate next word>
                current_input = top_indices
            else:
                current_input = gtruth[t]
        return outputs, hidden

Сам класс `Seq2Seq` будет просто прятать внутри себя энкодер и декодер, запуская их поочереди:

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, embedding_dim, rnn_size, input_size, output_size, encoder_n_layers, decoder_n_layers, dropout,
                 max_length, use_cuda):
        super(Seq2Seq, self).__init__()

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.output_size = output_size
        self.rnn_size = rnn_size
        self.encoder_n_layers = encoder_n_layers
        self.decoder_n_layers = decoder_n_layers
        self.dropout = dropout
        self.max_length = max_length
        self.use_cuda = use_cuda
        
        self.encoder = EncoderRNN(self.input_size, embedding_dim, rnn_size, dropout=dropout,
                                  n_layers=encoder_n_layers)
        self.decoder = DecoderRNN(embedding_dim, rnn_size, self.output_size, dropout=dropout,
                                  max_length=max_length, n_layers=decoder_n_layers, use_cuda=use_cuda)

    def forward(self, variable, sos_index, gtruth=None):
        encoder_output, encoder_hidden = self.encoder.forward(variable)
        current_input = self.decoder.init_state(variable.size(1), sos_index)
        max_length = self.max_length
        if gtruth is not None:
            max_length = min(self.max_length, gtruth.size(0))
        decoder_output, _ = self.decoder.forward(current_input, encoder_hidden, max_length, gtruth)

        return encoder_output, decoder_output

In [0]:
use_cuda = torch.cuda.is_available()
model = Seq2Seq(embedding_dim=200, 
                rnn_size=200, 
                input_size=src_vocabulary.size(),
                output_size=tgt_vocabulary.size(), 
                encoder_n_layers=2, 
                decoder_n_layers=2, 
                dropout=0.3, 
                max_length=10, 
                use_cuda=use_cuda)
model = model.cuda() if use_cuda else model
print(model)
params = sum([np.prod(p.size()) for p in model.parameters()])
print("Params: ", params)

**Задание** Допишите функцию для обучения модели.

In [0]:
main_optimizer = optim.Adam(model.parameters(), lr=0.003)

weight = torch.ones(tgt_vocabulary.size())
weight[tgt_vocabulary.get_pad()] = 0
weight = weight.cuda() if use_cuda else weight
criterion = nn.NLLLoss(weight, size_average=False)

print_every = 100
loss_average = 0
big_epochs = 3
for big_epoch in range(big_epochs):
    for epoch, (src_batch, tgt_batch) in enumerate(batch_generator):
        src_batch = src_batch.cuda() if use_cuda else src_batch
        tgt_batch = tgt_batch.cuda() if use_cuda else tgt_batch
        <your code here>
    loss_average = 0

Перевод для простоты будет делать так: на каждом шаге выбирать слово с наибольшей вероятностью.

При этом более правильным будет использовать beam search: на каждом шаге выбирать несколько вариантов продолжений текущих лучших последовательностей:  
![](https://4.bp.blogspot.com/-Jjpb7iyB37A/WBZI4ImGQII/AAAAAAAAA9s/ululnUWt2vw9NMKuEr-F9H8tR2LEv36lACLcB/s1600/prefix_probability_tree.png)   
From [Using beam search to generate the most probable sentence](https://geekyisawesome.blogspot.ru/2016/10/using-beam-search-to-generate-most.html).

In [0]:
def translate(model, sentence, src_vocabulary, tgt_vocabulary):
    model.eval()
    tokens = [token.lower() for token in word_tokenize(sentence.strip())]
    indices = src_vocabulary.get_indices(" ".join(tokens))
    variable = autograd.Variable(torch.LongTensor(indices))
    variable = variable.unsqueeze(1)
    variable = variable.cuda() if use_cuda else variable
    _, output = model.forward(variable, tgt_vocabulary.get_sos())
    result = []
    for t in range(output.size(0)):
        top_indices = output[t].topk(1, dim=1)[1].view(-1)
        index = top_indices.data[0]
        if index == tgt_vocabulary.get_eos():
            break
        result.append(tgt_vocabulary.get_word(index))
    return " ".join(result)

translate(model, "Тебя зовут Том.", src_vocabulary, tgt_vocabulary)

Стандартным способом оценки качества модели является [BLEU метрика](https://machinelearningmastery.com/calculate-bleu-score-for-text-python/).

In [0]:
!wget https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

In [0]:
with open(SRC_VAL_FILENAME, "r", encoding='utf-8') as f, open("pred.txt", "w", encoding='utf-8') as w:
    i = 0
    for line in f:
        w.write(translate(model, line, src_vocabulary, tgt_vocabulary)+"\n")
        if i % 500 == 0:
            print(i)
        i += 1

In [0]:
!perl multi-bleu.perl en.val.clean.txt < pred.txt

## Attention

Одна из основных проблем того, что мы делали до сих пор хорошо формулируется так:
```
"You can't cram the meaning of a whole %&!$ing sentence into a single $&!*ing vector!"
(c) Prof. Ray Mooney
```

Давайте запоминать все скрытые состояния энкодера, а не только последнее.

Дальше, для вычисления нового слова при генерации найдем сначала представление уже сгенерированного контекста (по которому обычно и генерируется следующее слово).  
По этому представлению посчитаем оценки полезности состояний энкодера: `attention weights` на картинке ниже. Чем выше вес - тем более полезно состояние. (Можно, кстати, представлять, что в предыдущем варианте мы просто давали всем состояниям кроме последнего вес 0, а последнему - 1).

С этими весами состояния энкодера суммируются, и мы получаем взвешенный вектор-представление контекста. Опять вектор?! Но теперь этот вектор получен для конкретного генерируемого слова - это же гораздо лучше, чем пытаться сделать один вектор сразу для всех генерируемых слов.

![attention](https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg =x400)  
From [Neural Machine Translation (seq2seq) Tutorial](https://www.tensorflow.org/tutorials/seq2seq).

В результате получаются такие красивые картинки с визуализацией аттеншена:   
![att-vis](https://www.tensorflow.org/images/seq2seq/attention_vis.jpg =x500)

Яркость ячейки показывает насколько много внимания уделяла модель данному слову на исходном языке при генерации соответствующего ему слова.

Очень красивая статья с демонстрацией attention'а: [Attention and Augmented Recurrent Neural Networks](https://distill.pub/2016/augmented-rnns/).

---

В общем случае, attention работает так: пусть у нас есть набор скрытых состояний $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - представлений слов из исходного языка, полученных с помощью энкодера. И есть некоторое текущее скрытое состояние $\mathbf{h}_i$ - скажем, представление, используемое для предсказания слова на нужном нам языке.

Тогда с помощью аттеншена мы можем получить взвешенное представление контекста $\mathbf{s}_1, \ldots, \mathbf{s}_m$ - вектор $c_i$:
$$
\begin{align}\begin{split}
\mathbf{c}_i &= \sum\limits_j a_{ij}\mathbf{s}_j\\
\mathbf{a}_{ij} &= \text{softmax}(f_{att}(\mathbf{h}_i, \mathbf{s}_j))
\end{split}\end{align}
$$

$f_{att}$ - функция, которая говорит, насколько хорошо $\mathbf{h}_i$ и $\mathbf{s}_j$ подходят друг другу.

Самые популярные её варианты:
- Additive attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{v}_a{}^\top \text{tanh}(\mathbf{W}_a[\mathbf{h}_i; \mathbf{s}_j])$$
- Dot attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{s}_j$$
- Multiplicative attention:
$$f_{att}(\mathbf{h}_i, \mathbf{s}_j) = \mathbf{h}_i^\top \mathbf{W}_a \mathbf{s}_j$$

----

Напишем реализацию простенького attention'а.

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_size, output_size, max_length, n_layers=3, 
                 dropout=0.3, use_cuda=False, use_attention=True):
        super(AttnDecoderRNN, self).__init__()

        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.use_cuda = use_cuda
        self.max_length = max_length
        self.use_attention = use_attention

        self.embedding = nn.Embedding(output_size, embedding_dim)
        
        if self.use_attention:
            self.attn = nn.Linear(hidden_size + embedding_dim, self.max_length, bias=False)
            self.attn_sm = nn.Softmax(dim=1)
            self.attn_out = nn.Linear(hidden_size + embedding_dim, embedding_dim, bias=False)
            self.attn_out_relu = nn.ReLU()
        
        self.rnn = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout)
        self.generator = Generator(hidden_size, output_size)
        
    def step(self, batch_input, hidden, encoder_output):
        # batch_input: B
        # hidden: (n_layers x B x N, n_layers x B x N)
        # encoder_output: L x B x N
        # output: 1 x B x N
        # embedded:  B x E
        # attn_weights: B x 1 x L
        # context: B x 1 x N
        # rnn_input: B x N
        
        embedded = self.embedding(batch_input)
        
        attn_weights = None
        if self.use_attention:
            attn_weights = self.attn_sm(self.attn(torch.cat((embedded, hidden[0][-1]), 1))).unsqueeze(1)
            max_length = encoder_output.size(0)
            context = torch.bmm(attn_weights[:, :, :max_length], encoder_output.transpose(0, 1))
            rnn_input = torch.cat((embedded, context.squeeze(1)), 1)
            rnn_input = self.attn_out_relu(self.attn_out(rnn_input))
        else:
            rnn_input = embedded
        output, hidden = self.rnn(rnn_input.unsqueeze(0), hidden)
        return output, hidden, attn_weights[:, :, :max_length].transpose(0, 1).squeeze(0)

    def init_state(self, batch_size, sos_index):
        initial_input = autograd.Variable(torch.zeros((batch_size,)).type(torch.LongTensor))
        initial_input = torch.add(initial_input, sos_index)
        initial_input = initial_input.cuda() if self.use_cuda else initial_input
        return initial_input

    def forward(self, current_input, hidden, length, encoder_output, gtruth=None):
        outputs = autograd.Variable(torch.zeros(length, current_input.size(0), self.output_size))
        outputs = outputs.cuda() if self.use_cuda else outputs
        
        attn_outputs = autograd.Variable(torch.zeros(length, current_input.size(0), encoder_output.size(0)))
        attn_outputs = attn_outputs.cuda() if self.use_cuda else attn_outputs

        for t in range(length):
            output, hidden, attn_weights = self.step(current_input, hidden, encoder_output)
            attn_outputs[t] = attn_weights
            scores = self.generator.forward(output.squeeze(0))
            outputs[t] = scores
            if gtruth is None:
                <your code here>
                current_input = top_indices
            else:
                current_input = gtruth[t]
        return outputs, hidden, attn_outputs

In [0]:
class AttnSeq2Seq(nn.Module):
    def __init__(self, embedding_dim, rnn_size, input_size, output_size, encoder_n_layers, decoder_n_layers, dropout,
                 max_length, use_cuda, bidirectional, use_attention=True):
        super(AttnSeq2Seq, self).__init__()

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.output_size = output_size
        self.rnn_size = rnn_size
        self.encoder_n_layers = encoder_n_layers
        self.decoder_n_layers = decoder_n_layers
        self.dropout = dropout
        self.max_length = max_length
        self.use_cuda = use_cuda
        self.bidirectional = bidirectional
        self.use_attention = use_attention

        self.encoder = EncoderRNN(self.input_size, embedding_dim, rnn_size, dropout=dropout,
                                  n_layers=encoder_n_layers, bidirectional=bidirectional)
        self.decoder = AttnDecoderRNN(embedding_dim, rnn_size, self.output_size, dropout=dropout,
                                      max_length=max_length, n_layers=decoder_n_layers, use_cuda=use_cuda, 
                                      use_attention=use_attention)

    def forward(self, variable, sos_index, gtruth=None):
        encoder_output, encoder_hidden = self.encoder.forward(variable)
        current_input = self.decoder.init_state(variable.size(1), sos_index)
        max_length = self.max_length
        if gtruth is not None:
            max_length = min(self.max_length, gtruth.size(0))
        decoder_output, _, attn_output = \
            self.decoder.forward(current_input, encoder_hidden, 
                                 max_length, encoder_output, gtruth)

        return encoder_output, decoder_output, attn_output

In [0]:
use_cuda = torch.cuda.is_available()
model = AttnSeq2Seq(embedding_dim=300, 
                rnn_size=300, 
                input_size=src_vocabulary.size(),
                output_size=tgt_vocabulary.size(), 
                encoder_n_layers=2, 
                decoder_n_layers=2, 
                dropout=0.3, 
                max_length=10, 
                use_cuda=use_cuda, 
                bidirectional=False)
model = model.cuda() if use_cuda else model
print(model)

In [0]:
main_optimizer = optim.Adam(model.parameters(), lr=0.003)
weight = torch.ones(tgt_vocabulary.size())
weight[tgt_vocabulary.get_pad()] = 0
weight = weight.cuda() if use_cuda else weight
criterion = nn.NLLLoss(weight, size_average=False)
print_every = 100
loss_average = 0
big_epochs = 3
for big_epoch in range(big_epochs):
    for epoch, (src_batch, tgt_batch) in enumerate(batch_generator):
        src_batch = src_batch.cuda() if use_cuda else src_batch
        tgt_batch = tgt_batch.cuda() if use_cuda else tgt_batch
        <your code here>
    loss_average = 0

Попробуем тоже визуализировать attention!

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def show_attention(input_tokens, output_tokens, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().data.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([""] + input_tokens, rotation=90)
    ax.set_yticklabels([""] + output_tokens)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [0]:
def translate(model, sentence, src_vocabulary, tgt_vocabulary):
    model.eval()
    tokens = [token.lower() for token in word_tokenize(sentence.strip())]
    indices = src_vocabulary.get_indices(" ".join(tokens))
    variable = autograd.Variable(torch.LongTensor(indices))
    variable = variable.unsqueeze(1)
    variable = variable.cuda() if use_cuda else variable
    _, output, attn = model.forward(variable, tgt_vocabulary.get_sos())
    attn = attn.squeeze(1)
    result = []
    for t in range(output.size(0)):
        top_indices = output[t].topk(1, dim=1)[1].view(-1)
        index = top_indices.data[0]
        if index == tgt_vocabulary.get_eos():
            break
        result.append(tgt_vocabulary.get_word(index))
    show_attention(tokens, result, attn)
    return " ".join(result)

translate(model, "Тебя зовут Том.", src_vocabulary, tgt_vocabulary)

**Задание** Реализуйте Additive attention и Multiplicative attention.